In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from __future__ import absolute_import, division, print_function

import tensorflow_hub as hub
import tensorflow_text as hub_text  # A dependency of the preprocessing model
from official import nlp
import official.nlp.optimization
import itertools
import numpy as np
import tensorflow as tf
import pandas as pd
import time
from fastprogress import master_bar, progress_bar
import math
import official.nlp.bert.tokenization
from official.nlp import bert
from nltk.tokenize import word_tokenize
import string 
import re
tf.get_logger().setLevel('ERROR')


def sentence_token_taging(test_sentence_tags, tokenized_sentences, regulator_ds,regulated_ds):
    entity = ''
    num_entities = 0
    n = 0
    TF_Regulator, RegulatedGene = [],[]
    FinalEntities, temp,temp_s,NerSentence = [],[],[],[]
    for num in (range(len(test_sentence_tags))): 
        for num_word, (entity_tags, words) in enumerate(zip(test_sentence_tags[num], tokenized_sentences[num])):
            if entity_tags.startswith('B'):
                entity += '[SEP] ' + str(words) + ' '
                num_entities += 1
                temp_s.append('GENE')
                
            if entity_tags.startswith('I'):
                if test_sentence_tags[num][num_word-1].startswith('O'):
                    entity += '[SEP] ' + str(words) + ' '
                    num_entities += 1
                    temp_s.append('GENE')
                       
                else:
                    entity += str(words) + ' '
            
            if entity_tags.startswith('B') == False and entity_tags.startswith('I') == False:
                temp_s.append(words)
    
        if entity != '':
            temp.append(entity.split('[SEP] ')[1:])        
        FinalEntities.append(temp)
        entity = ''
        temp_str = " ".join(temp_s).replace('  ',' ')
        NerSentence.append(temp_str.split(' '))
        temp_s = []
        temp = []
        TF_Regulator.append(regulator_ds[num])
        RegulatedGene.append(regulated_ds[num])

        
    print(f'Completed. Found {num_entities} genes.')
    return [NerSentence,TF_Regulator,RegulatedGene,FinalEntities]


In [2]:
preprocessor = hub.load( "https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3")
# ris_eco = pd.read_csv('../DatasetNER/STM-ris-curadas.csv').fillna(0)
# eco_sentence = list(ris_eco['SENTENCE'])
# eco_regulator = list(ris_eco['TF'])
# eco_regulated = list(ris_eco['GENE/TU'])

ris_eco = pd.read_csv('../DatasetNER/ECO-ris-curadas.csv').fillna(0)
eco_sentence = ris_eco['SENTENCE']
eco_regulator = list(ris_eco['REGULATOR'])
eco_effect =  list(ris_eco['EFFECT'])


eco_regulated = []
for index in range(len(eco_regulator)):
    if list(ris_eco['REGULATED_GENE'])[index] == 0:
        eco_regulated.append(list(ris_eco['REGULATED_TU'])[index])
    else:
        eco_regulated.append(list(ris_eco['REGULATED_GENE'])[index])


raw_text = preprocessor(eco_sentence)
bert_classifier = tf.keras.models.load_model('../NERModel')
prediction = bert_classifier.predict(raw_text)
tokenizer = bert.tokenization.FullTokenizer('../vocabNER.txt', do_lower_case=False)

pre_txt = []
for indx in range(len(eco_sentence)):
    pre_txt.append(' '.join(tokenizer.tokenize(eco_sentence[indx])).replace(' ##','').replace('##','').replace('  ',' ').split(' '))
    
sentence_tags = []
raw_sentences = []
TF_Regulator_, RegulatedGene_ = [],[]
for i, sentence in enumerate(prediction):
    temp_rel = []
    for n_wor, pred_word in enumerate(sentence):
        val = list(pred_word)
        if val[1] == max(val):
            temp_rel.append('O')
        elif val[2] == max(val):
            temp_rel.append('B-GENE')
        elif val[3] == max(val):
            temp_rel.append('I-GENE')
    if len(pre_txt[i]) == len(temp_rel):
        raw_sentences.append(pre_txt[i])
        sentence_tags.append(temp_rel)
        RegulatedGene_.append(eco_regulated[i])
        TF_Regulator_.append(eco_regulator[i])

52/52 [==============================] - 315s 6s/step


In [3]:
ner_sentence,tf_regulator, regulated_gene, final_entities  = sentence_token_taging(sentence_tags,raw_sentences, TF_Regulator_, RegulatedGene_)


output_sentence = []
for line in range(len(ner_sentence)):
    new_line = ' '.join(ner_sentence[line])
    genes_in_sentence = 0
    prducts_in_sentence = 0
    if final_entities[line]:
        for x in range(len(final_entities[line][0])):
            if genes_in_sentence == 0:
                en = final_entities[line][0][x].replace(TF_Regulator_[line].split(' ')[0],'RegProtein',1)
                if 'RegProtein' in en:
                    new_line = new_line.replace('GENE',f'[ {en}]',1).replace('  ',' ').replace('[ ','[E1]').replace(' ]','[/E1]')
                    genes_in_sentence += 1
            else:
                new_line = new_line.replace('GENE',final_entities[line][0][x],1).replace('  ',' ')

            if prducts_in_sentence == 0:
                te = final_entities[line][0][x].replace(RegulatedGene_[line].split(' ')[0],'GeneReg',1)
                if 'GeneReg' in te:
                    new_line = new_line.replace('GENE',f'[ {te}]',1).replace('  ',' ').replace('[ ','[E2]').replace(' ]','[/E2]')
                    prducts_in_sentence += 1
            else:
                new_line = new_line.replace('GENE',final_entities[line][0][x],1).replace('  ',' ')
                    
            if genes_in_sentence + prducts_in_sentence <= 1:
                if ('RegProtein' != en.split(' ')[1]) and ('GeneReg' not in te):
                    new_line = new_line.replace('GENE',final_entities[line][0][x],1).replace('  ',' ')
                    
        if '[/E2]' in new_line and '[/E1]' in new_line:
            output_sentence.append(new_line.replace('  ',' '))
        
# input_sentences = list(set(output_sentence))

input_sentences = []
for e,x in enumerate(output_sentence):
    if x[0] == ' ':
        output_sentence[e] = output_sentence[e][1:]
    temp = ''
    list_rel = []
    n = 0
    for i, word in enumerate(x.split(' ')):
        temp += word + ' '
        if n == 2:
            list_rel[-1] += word + ' '
        if '[/E1]' in word :
            list_rel.append(temp)
            if n < 1:
                temp = ''
            n += 1
        if '[/E2]' in word:
            list_rel.append(temp)
            if n < 1:
                temp = ''
            n += 1
    element = '[SEP] '.join(list_rel).replace('[/E1]','').replace('[/E2]','').replace('[E1]','').replace('[E2]','').replace('  ','')
    input_sentences.append(element)
    

labels = []
sentences = []
for indx in range(len(input_sentences)):
    temp_label = []
    n = 0
    for word in input_sentences[indx].replace('  ',' ').split(' '):
        if n == 1 and '[SEP]' not in word and word != 'GeneReg' and word != 'RegProtein':
            temp_label.append('I-Rel')
        if '[SEP]' in word:
            n = 1
            temp_label.append('0')
        if (word == 'GeneReg' or word == 'RegProtein') and n == 1:
            n = 2
            #temp_label.append('O')
        #else:
        if n == 0:
            temp_label.append('O')
        if n == 2:
            temp_label.append('O')
    sentences.append(input_sentences[indx].replace('  ',' ').split(' [SEP]'))
    labels.append(temp_label)
    
with open('../DatasetRE/test_mnli.txt','w+') as file:
    for i, line in enumerate(sentences):
        for e,segment in enumerate(line):
            file.write(segment)
            file.write('\t')
        for tag in labels[i]:
            file.write(str(tag))
            file.write(' ')
        file.write('\n')


Completed. Found 7628 genes.
